In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [4]:
df = pd.read_csv('../dataset/fake_job_postings.csv')
df.drop(columns=['job_id'],inplace=True)
df.replace()
df.fillna('-', inplace = True)
df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,Marketing Intern,"US, NY, New York",Marketing,-,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,-,0,1,0,Other,Internship,-,-,Marketing,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,-,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,-,Marketing and Advertising,Customer Service,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",-,-,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,-,0,1,0,-,-,-,-,-,0
3,Account Executive - Washington DC,"US, DC, Washington",Sales,-,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,Bill Review Manager,"US, FL, Fort Worth",-,-,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


### Tokenização e Lemmatization

In [5]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JoaoFelipeMoraisVasc\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JoaoFelipeMoraisVasc\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JoaoFelipeMoraisVasc\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text) # remove tags html
    text = re.sub(r'http\S+', '', text) # remove url
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

In [7]:
long_string_columns = ['title', 'location', 'department', 'company_profile', 'description', 'requirements', 'benefits', 'industry', 'function']

In [8]:
for column in long_string_columns:
    df[column + '_words'] = df[column].apply(preprocess_text)


df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,...,fraudulent,title_words,location_words,department_words,company_profile_words,description_words,requirements_words,benefits_words,industry_words,function_words
0,Marketing Intern,"US, NY, New York",Marketing,-,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,-,0,1,...,0,"[marketing, intern]","[u, ny, new, york]",[marketing],"[food52, created, groundbreaking, award, winni...","[food52, fast, growing, james, beard, award, w...","[experience, content, management, system, majo...",[],[],[marketing]
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,-,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,...,0,"[customer, service, cloud, video, production]","[nz, auckland]",[success],"[90, second, world, cloud, video, production, ...","[organised, focused, vibrant, awesome, passion...","[expect, key, responsibility, communicate, cli...","[get, usthrough, part, 90, second, team, gain,...","[marketing, advertising]","[customer, service]"
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",-,-,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,-,0,1,...,0,"[commissioning, machinery, assistant, cma]","[u, ia, wever]",[],"[valor, service, provides, workforce, solution...","[client, located, houston, actively, seeking, ...","[implement, pre, commissioning, commissioning,...",[],[],[]
3,Account Executive - Washington DC,"US, DC, Washington",Sales,-,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,...,0,"[account, executive, washington, dc]","[u, dc, washington]",[sale],"[passion, improving, quality, life, geography,...","[company, esri, environmental, system, researc...","[education, bachelor, master, gi, business, ad...","[culture, anything, corporate, collaborative, ...","[computer, software]",[sale]
4,Bill Review Manager,"US, FL, Fort Worth",-,-,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,...,0,"[bill, review, manager]","[u, fl, fort, worth]",[],"[spotsource, solution, llc, global, human, cap...","[job, title, itemization, review, managerlocat...","[qualification, rn, license, state, texasdiplo...","[full, benefit, offered]","[hospital, health, care]","[health, care, provider]"


In [9]:
df.drop(columns=long_string_columns,inplace=True)

### Coluna Range

In [10]:
df['salary_range'].unique()

array(['-', '20000-28000', '100000-120000', '120000-150000',
       '50000-65000', '40000-50000', '60-80', '65000-70000', '75-115',
       '75000-110000', '17000-20000', '16000-28000', '95000-115000',
       '15000-18000', '50000-70000', '45000-60000', '30000-40000',
       '70000-90000', '10000-14000', '50-110', '28000-45000', '0-34300',
       '35000-40000', '9-Dec', '44000-57000', '18500-28000',
       '55000-75000', '30000-35000', '0-0', '20000-40000',
       '360000-600000', '50000-80000', '80000-100000', '52000-78000',
       '15750-15750', '40000-65000', '45000-50000', '30000-37000',
       '45000-67000', '35000-100000', '180000-216000', '45000-65000',
       '28000-32000', '0-1000', '36000-40000', '80000-110000',
       '35000-73000', '19000-19000', '60000-120000', '120000-15000000',
       '42000-55000', '90000-120000', '100000-150000', '28000-38000',
       '1600-1700', '50000-60000', '30000-70000', '32000-40000', '50-100',
       '9000-17000', '23040-28800', '105-110', '1300

In [11]:
def validate_and_split_salary_range(salary_range):
    if salary_range == 0:
        return 0, 0
    # Expressão regular para validar o formato xxxx-yyyy
    pattern = r'^\d+-\d+$'
    if re.match(pattern, salary_range):
        salary_min, salary_max = salary_range.split('-')
        return int(salary_min), int(salary_max)
    else:
        return 0, 0

# Aplicar a função para a coluna salary_range
df[['salary_min', 'salary_max']] = df['salary_range'].apply(lambda x: pd.Series(validate_and_split_salary_range(x)))


In [12]:
df.drop(columns=['salary_range'],inplace=True)

In [13]:
df.head()

,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent,title_words,location_words,department_words,company_profile_words,description_words,requirements_words,benefits_words,industry_words,function_words,salary_min,salary_max
0,0,1,0,Other,Internship,-,0,"[marketing, intern]","[u, ny, new, york]",[marketing],"[food52, created, groundbreaking, award, winni...","[food52, fast, growing, james, beard, award, w...","[experience, content, management, system, majo...",[],[],[marketing],0,0
1,0,1,0,Full-time,Not Applicable,-,0,"[customer, service, cloud, video, production]","[nz, auckland]",[success],"[90, second, world, cloud, video, production, ...","[organised, focused, vibrant, awesome, passion...","[expect, key, responsibility, communicate, cli...","[get, usthrough, part, 90, second, team, gain,...","[marketing, advertising]","[customer, service]",0,0
2,0,1,0,-,-,-,0,"[commissioning, machinery, assistant, cma]","[u, ia, wever]",[],"[valor, service, provides, workforce, solution...","[client, located, houston, actively, seeking, ...","[implement, pre, commissioning, commissioning,...",[],[],[],0,0
3,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,0,"[account, executive, washington, dc]","[u, dc, washington]",[sale],"[passion, improving, quality, life, geography,...","[company, esri, environmental, system, researc...","[education, bachelor, master, gi, business, ad...","[culture, anything, corporate, collaborative, ...","[computer, software]",[sale],0,0
4,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0,"[bill, review, manager]","[u, fl, fort, worth]",[],"[spotsource, solution, llc, global, human, cap...","[job, title, itemization, review, managerlocat...","[qualification, rn, license, state, texasdiplo...","[full, benefit, offered]","[hospital, health, care]","[health, care, provider]",0,0


### Colunas categóricas

In [14]:
category_columns = ['employment_type', 'required_experience', 'required_education']
df_encoded = pd.get_dummies(df, columns=category_columns)

In [15]:
df_encoded.columns

Index(['telecommuting', 'has_company_logo', 'has_questions', 'fraudulent',
       'title_words', 'location_words', 'department_words',
       'company_profile_words', 'description_words', 'requirements_words',
       'benefits_words', 'industry_words', 'function_words', 'salary_min',
       'salary_max', 'employment_type_-', 'employment_type_Contract',
       'employment_type_Full-time', 'employment_type_Other',
       'employment_type_Part-time', 'employment_type_Temporary',
       'required_experience_-', 'required_experience_Associate',
       'required_experience_Director', 'required_experience_Entry level',
       'required_experience_Executive', 'required_experience_Internship',
       'required_experience_Mid-Senior level',
       'required_experience_Not Applicable', 'required_education_-',
       'required_education_Associate Degree',
       'required_education_Bachelor's Degree',
       'required_education_Certification', 'required_education_Doctorate',
       'required_educa

### Vetorização

In [16]:
from gensim.models import Word2Vec
sentences = []
for column in long_string_columns:
    sentences += df_encoded[column+'_words'].tolist()

# Treinar o modelo Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [17]:
import numpy as np

def words_to_vector(words, model):
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:  # Se não houver palavras reconhecidas no modelo
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Aplicar a função para cada coluna de listas de palavras
for column in long_string_columns:
    df_encoded[column +'_vector'] = df[column + "_words"].apply(lambda x: words_to_vector(x, model))
    df_encoded.drop(columns=[column+'_words'], inplace=True)

In [18]:
df_encoded.dtypes

telecommuting                                            int64
has_company_logo                                         int64
has_questions                                            int64
fraudulent                                               int64
salary_min                                               int64
salary_max                                               int64
employment_type_-                                        uint8
employment_type_Contract                                 uint8
employment_type_Full-time                                uint8
employment_type_Other                                    uint8
employment_type_Part-time                                uint8
employment_type_Temporary                                uint8
required_experience_-                                    uint8
required_experience_Associate                            uint8
required_experience_Director                             uint8
required_experience_Entry level                        

In [19]:
def calcular_media(vetor):
    return np.mean(vetor)

# Aplicar a função para calcular a média em todas as colunas do DataFrame
for coluna in long_string_columns:
    df_encoded[coluna + "_vector"] = df_encoded[coluna + "_vector"].apply(calcular_media)

In [20]:
df_encoded.head()

,telecommuting,has_company_logo,has_questions,fraudulent,salary_min,salary_max,employment_type_-,employment_type_Contract,employment_type_Full-time,employment_type_Other,...,required_education_Vocational - HS Diploma,title_vector,location_vector,department_vector,company_profile_vector,description_vector,requirements_vector,benefits_vector,industry_vector,function_vector
0,0,1,0,0,0,0,0,0,0,1,...,0,0.247668,0.115215,0.244362,-0.009132,0.035534,0.031247,0.000000,0.000000,0.244362
1,0,1,0,0,0,0,0,0,1,0,...,0,-0.060168,0.102202,0.254773,-0.012521,0.004112,0.006893,0.013835,0.113106,-0.101499
2,0,1,0,0,0,0,1,0,0,0,...,0,0.069010,0.002190,0.000000,-0.023828,0.008513,-0.020861,0.000000,0.000000,0.000000
3,0,1,0,0,0,0,0,0,1,0,...,0,0.048156,0.061260,0.092472,0.000941,0.023731,0.036613,0.038574,-0.072177,0.092472
4,0,1,1,0,0,0,0,0,1,0,...,0,0.002881,0.038611,0.000000,-0.024219,-0.018626,0.028890,0.016103,-0.115658,-0.178535


In [21]:
import pickle
with open('../dataset/fake_job_word2vec.pkl', 'wb') as f:
    pickle.dump(df_encoded, f)